In [1]:
import pandas as pd
#ignore warnings

In [2]:
df = pd.read_csv('data/ibtracs.since1980.list.v04r00.csv', dtype='object', parse_dates=True, skiprows=[1])
#drop first row as it's a multi index

pd.set_option('display.max_columns', None)
df.head(3)

The size of the file is really large but it will get smaller throughout the cleaning process.

In [4]:
df.shape

(271883, 163)

There are 163 columns and they are all reading in as object datatypes. I'll need to go through and clean these up.

In [5]:
df.columns = [x.lower() for x in df.columns]
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271883 entries, 0 to 271882
Data columns (total 163 columns):
sid                 object
season              object
number              object
basin               object
subbasin            object
name                object
iso_time            object
nature              object
lat                 object
lon                 object
wmo_wind            object
wmo_pres            object
wmo_agency          object
track_type          object
dist2land           object
landfall            object
iflag               object
usa_agency          object
usa_atcf_id         object
usa_lat             object
usa_lon             object
usa_record          object
usa_status          object
usa_wind            object
usa_pres            object
usa_sshs            object
usa_r34_ne          object
usa_r34_se          object
usa_r34_sw          object
usa_r34_nw          object
usa_r50_ne          object
usa_r50_se          object
usa_r50_sw          obje

The dataset has readings for storms at multiple points in their progression. There are 4,458 unique storms tracked.

In [6]:
#there are this many separate storms
df['sid'].nunique()

4458

My classification task will be to identify whether they are minor storms or severe Tropical Storms. Looking at my target column, nature, I can see six different classes that I need to clean up. 

NR, not reported, and MX, mixture will be removed as they don't tell me anything. TS, tropical storm, will be my '1' - a severe storm. ET, DS, and SS are extratropical, disturbance, and subtropical - less severe storms. These will be my '0' class. 

In [7]:
df['nature'].unique()

array(['TS', 'NR', 'ET', 'MX', 'SS', 'DS'], dtype=object)

In [8]:
df.drop(df.loc[df['nature'] == 'NR'].index, inplace=True)
df.drop(df.loc[df['nature'] == 'MX'].index, inplace=True)

In [9]:
#new column  
df['target'] = 0

# loop through the data and input a 1 where the storm is a Tropical storm
for row in df.index:
    if df['nature'][row] == 'TS':
        df['target'][row] = 1

C:\Users\christina.sarver\Anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [10]:
df['target'].value_counts(normalize=True)

1    0.897215
0    0.102785
Name: target, dtype: float64

So there is a pretty severe class imbalance here. Before I address that, I want to define my features. Taking a preliminary look through the columns in the dataframe and the documentation as a guide, I'm selecting the following as potential features to explore. 

In [11]:
initial_feats = ['sid', 'season', 'basin', 'subbasin', 'iso_time', 'lat', 'lon', 
                 'wmo_wind', 'dist2land', 'ds824_wind', 'td9636_stage', 'storm_speed', 'storm_dir']

In [12]:
xs_df = df[initial_feats]
xs_df.head()

,sid,season,basin,subbasin,iso_time,lat,lon,wmo_wind,dist2land,ds824_wind,td9636_stage,storm_speed,storm_dir
0,1980001S13173,1980,SP,MM,1980-01-01 00:00:00,-12.5000,172.500,,647,25,1,6,351
1,1980001S13173,1980,SP,MM,1980-01-01 03:00:00,-12.1927,172.441,,653,25,1,6,351
2,1980001S13173,1980,SP,MM,1980-01-01 06:00:00,-11.9144,172.412,,670,25,1,5,358
3,1980001S13173,1980,SP,MM,1980-01-01 09:00:00,-11.6863,172.435,,682,25,1,4,12
4,1980001S13173,1980,SP,MM,1980-01-01 12:00:00,-11.5000,172.500,,703,25,1,4,22


I'm able to convert some of the columns to workable datatypes. Some, that I've listed as 'remaining columns' ran an error as they have empty strings. For now I'm filling those with 0 and then I am able to convert them. 

In [17]:
xs_df[['lat', 'lon', 'dist2land']] = xs_df[['lat', 'lon', 'dist2land']].apply(pd.to_numeric)
xs_df[['iso_time', 'season']] = xs_df[['iso_time', 'season']].apply(pd.to_datetime)
xs_df['season'] = xs_df['season'].dt.year

C:\Users\christina.sarver\Anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
string_cols = ['sid', 'basin', 'subbasin']

In [19]:
remaining_cols = ['wmo_wind', 'ds824_wind', 'td9636_stage', 'storm_speed', 'storm_dir']
xs_df[remaining_cols] = xs_df[remaining_cols].apply(lambda x: x.str.replace(' ', '0'))
xs_df[remaining_cols] = xs_df[remaining_cols].apply(pd.to_numeric)

In [20]:
xs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226423 entries, 0 to 271072
Data columns (total 13 columns):
sid             226423 non-null object
season          226423 non-null int64
basin           190696 non-null object
subbasin        197687 non-null object
iso_time        226423 non-null datetime64[ns]
lat             226423 non-null float64
lon             226423 non-null float64
wmo_wind        226423 non-null int64
dist2land       226423 non-null int64
ds824_wind      226423 non-null int64
td9636_stage    226423 non-null int64
storm_speed     226423 non-null int64
storm_dir       226423 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(7), object(3)
memory usage: 34.2+ MB


All of my non-object columns have been converted! There are still null values in the basin & subbasin columns so I'm going to inspect those to see what the best action would be.

In [21]:
y_df = df['target'].to_frame()
clean_df = pd.concat([xs_df, y_df], axis=1)
clean_df.head()

In [22]:
grouped_df = clean_df.groupby(['sid']).mean()
#use a .max() for groupby 
grouped_df['target'].value_counts()

From inspecting the mean value for the target column of my data grouped by storm ID, I can see that storms change class over time. Because of this, I'm not going to look at the readings by storm but instead as independent readings to see what conditions change a storm to be severe

In [28]:
clean_df.loc[clean_df['sid'] == '2020307N12250']

,sid,season,basin,subbasin,iso_time,lat,lon,wmo_wind,dist2land,ds824_wind,td9636_stage,storm_speed,storm_dir,target
270814,2020307N12250,2020,EP,MM,2020-11-03 12:00:00,12.7000,-114.500,0,1236,0,0,9,299,0
270815,2020307N12250,2020,EP,MM,2020-11-03 15:00:00,13.1324,-114.962,0,1216,0,0,14,316,0
270816,2020307N12250,2020,EP,MM,2020-11-03 18:00:00,13.7000,-115.500,0,1181,0,0,15,312,1
270817,2020307N12250,2020,EP,MM,2020-11-03 21:00:00,14.1225,-116.077,0,1175,0,0,14,305,1
270818,2020307N12250,2020,EP,MM,2020-11-04 00:00:00,14.5000,-116.700,0,1174,0,0,14,299,1
270819,2020307N12250,2020,EP,MM,2020-11-04 03:00:00,14.8251,-117.373,0,1194,0,0,14,299,1
270820,2020307N12250,2020,EP,MM,2020-11-04 06:00:00,15.2000,-118.000,0,1203,0,0,14,312,1
270821,2020307N12250,2020,EP,MM,2020-11-04 09:00:00,15.7486,-118.443,0,1193,0,0,14,323,1
270822,2020307N12250,2020,EP,MM,2020-11-04 12:00:00,16.3333,-118.900,0,1175,0,0,16,315,1
270823,2020307N12250,2020,EP,MM,2020-11-04 15:00:00,16.8479,-119.595,0,1177,0,0,16,305,1


In [ ]:
#look at wind columns 
#run model
#adjust for class imbalance